In [1]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from skimage.io import imshow,imread

first=imageio.imread('images/1.gif')
print(first)
#imageio.imwrite('images/tesla.jpeg',first[:,:])
second=imageio.imread('images/2.gif')
third=imageio.imread('images/3.gif')
fourth=imageio.imread('images/4.gif')
data=np.loadtxt("river.txt",dtype=np.int64,delimiter=",")
X=data[:,0:1]
Y=data[:,1:2]
data1=np.loadtxt("Nonriver.txt",dtype=np.int64,delimiter=",")
X2=data1[:,0:1]
Y2=data1[:,1:2]
print(X.shape)
print(X2.shape)

[[41 41 38 ... 41 41 41]
 [41 42 41 ... 44 41 41]
 [41 43 39 ... 42 40 40]
 ...
 [39 38 38 ... 38 39 37]
 [40 39 40 ... 38 39 37]
 [41 42 42 ... 38 38 37]]
(50, 1)
(100, 1)


In [2]:
def cal_cov(x,y):
    m=len(x)
    xmean=np.mean(x)
    ymean=np.mean(y)
    x=x-xmean
    y=y-ymean
    temp=np.transpose(x)*y
    temp=temp/m
    return temp[0,0]


##calculating covariance matrix for River dataset
red=[]
green=[]
blue=[]
intensity=[]
for i in range(0,len(X)):
    red.append(first[X[i],Y[i]])
    green.append(second[X[i],Y[i]])
    blue.append(third[X[i],Y[i]])
    intensity.append(fourth[X[i],Y[i]])
red=np.matrix(red)
green=np.matrix(green)
blue=np.matrix(blue)
intensity=np.matrix(intensity)
river_mean=np.matrix([np.mean(red),np.mean(green),np.mean(blue),np.mean(intensity)])
print("The river mean matrix is",river_mean)
rivercov_matrix=[[cal_cov(red,red),cal_cov(red,green),cal_cov(red,blue),cal_cov(red,intensity)],
   [cal_cov(green,red),cal_cov(green,green),cal_cov(green,blue),cal_cov(green,intensity)],
   [cal_cov(blue,red),cal_cov(blue,green),cal_cov(blue,blue),cal_cov(blue,intensity)],
   [cal_cov(intensity,red),cal_cov(intensity,green),cal_cov(intensity,blue),cal_cov(intensity,intensity)]]
rivercov_matrix=np.matrix(rivercov_matrix)
print("The covariance matrix for River is",rivercov_matrix)

##calculating covariance matrix for non-river dataset
red=[]
green=[]
blue=[]
intensity=[]
for i in range(0,len(X2)):
    red.append(first[X2[i],Y2[i]])
    green.append(second[X2[i],Y2[i]])
    blue.append(third[X2[i],Y2[i]])
    intensity.append(fourth[X2[i],Y2[i]])
red=np.matrix(red)
green=np.matrix(green)
blue=np.matrix(blue)
intensity=np.matrix(intensity)
nonriver_mean=np.matrix([np.mean(red),np.mean(green),np.mean(blue),np.mean(intensity)])
print("The nonriver mean matrix is",nonriver_mean)
nonrivercov_matrix=[[cal_cov(red,red),cal_cov(red,green),cal_cov(red,blue),cal_cov(red,intensity)],
   [cal_cov(green,red),cal_cov(green,green),cal_cov(green,blue),cal_cov(green,intensity)],
   [cal_cov(blue,red),cal_cov(blue,green),cal_cov(blue,blue),cal_cov(blue,intensity)],
   [cal_cov(intensity,red),cal_cov(intensity,green),cal_cov(intensity,blue),cal_cov(intensity,intensity)]]
nonrivercov_matrix=np.matrix(nonrivercov_matrix)
print("The covariance matrix for Non-River is",nonrivercov_matrix)

The river mean matrix is [[41.8  27.16 28.68 34.64]]
The covariance matrix for River is [[ 7.32    7.112   7.476   1.208 ]
 [ 7.112   8.2944  9.7712  2.6176]
 [ 7.476   9.7712 21.6976  6.6848]
 [ 1.208   2.6176  6.6848 32.1104]]
The nonriver mean matrix is [[40.32 25.75 26.3  35.26]]
The covariance matrix for Non-River is [[ 7.7376  7.04    9.154   2.2468]
 [ 7.04    7.2275  9.275   2.825 ]
 [ 9.154   9.275  14.89    2.582 ]
 [ 2.2468  2.825   2.582  29.5324]]


In [3]:
##creating test-data
outimage=[]
capitalp1=0.99
capitalp2=0.01
for i in range(0,512):
    temp=[]
    for j in range(0,512):
        test_data=[first[i,j],second[i,j],third[i,j],fourth[i,j]]
        test_data=np.matrix(test_data)
        test_data1=test_data-river_mean
        test_data2=test_data-nonriver_mean
        riverclass=test_data1*np.linalg.inv(rivercov_matrix)*np.transpose(test_data1)
        nonriverclass=test_data2*np.linalg.inv(nonrivercov_matrix)*np.transpose(test_data2)
        smallp1=-(0.5)*(1/np.sqrt(np.linalg.det(rivercov_matrix)))*np.exp(riverclass)
        smallp2=-(0.5)*(1/np.sqrt(np.linalg.det(nonrivercov_matrix)))*np.exp(nonriverclass)
        if((capitalp1*smallp1)>=(capitalp2*smallp2)):
            temp.append(255)
        else:
            temp.append(0)
    outimage.append(temp)

temp=outimage
outimage=np.uint8(np.array(outimage))
imageio.imwrite('images/sampleoutput.jpeg',outimage[:,:])

In [6]:
##calculating Accuracy
load_img=imageio.imread('images/PDCV1.jpeg')
count=0
for i in range(0,512):
    for j in range(0,512):
        if(load_img[i][j]==temp[i][j]):
            count+=1
accuracy=(count*100)/(512*512)
print("Accuracy achieved is",accuracy)

Accuracy achieved is 89.70184326171875
